In [99]:
#My default packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from dataclasses import dataclass
import dataclasses
import itertools
from heapq import heappop,heappush

from enum import Enum


# Day 24

In [100]:
@dataclass
class Point:
    x: int
    y: int

    def __hash__(self) -> int:
        return hash((self.x, self.y))

    def __add__(self, other) -> "Point":
        return Point(self.x + other.x, self.y + other.y)

    def __sub__(self, other) -> "Point":
        return Point(self.x - other.x, self.y - other.y)

    def __mod__(self,other) -> "Point":
        return Point(self.x%other.x,self.y%other.y)

    def __lt__(self,other):
        return self.x<other.x or (self.x==other.x and self.y<other.y)
    
    def __iter__(self):
        yield from dataclasses.asdict(self).values()


def Manhattan(p1,p2):
    return abs(p1.x-p2.x)+abs(p1.y-p2.y)



In [144]:
@dataclass
class Valley:
    north : set
    east : set
    south : set
    west : set
    dims : Point
    strval :list
    start : Point
    exit : Point

    @classmethod
    def fromfile(cls,filename):
        with open(filename) as f:
            N,E,S,W = set(),set(),set(),set()
            strval =[]
            start = Point(0,-1)
            l = f.readline().rstrip()
            Nx = len(l)-2
            j=0
            while len(l:=f.readline().rstrip().replace("#",""))>1:
                strval.append(l)
                # print(l)
                for i,c in enumerate(l):
                    match c:
                        case ">": E.add(Point(i,j))
                        case "<": W.add(Point(i,j))
                        case "v": S.add(Point(i,j))
                        case "^": N.add(Point(i,j))
                        case _: pass
                j+=1
            Ny = j
            exit = Point(Nx-1,Ny)
            return cls(N,E,S,W,Point(Nx,Ny),strval,start,exit)
    
    def toexit(self,pos):
        return Manhattan(pos,self.exit)

    def isfree(self,pos,t):
        if pos == self.start or pos == self.exit:
            return True
        if (pos+Point(0,t))%self.dims in self.north:
            return False
        elif (pos-Point(0,t))%self.dims in self.south:
            return False
        elif (pos-Point(t,0))%self.dims in self.east:
            return False
        elif (pos+Point(t,0))%self.dims in self.west:
            return False
        return True

    def inbounds(self,pos):
        if pos == self.start or pos == self.exit:
            return True
        elif 0<=pos.x and 0<= pos.y and pos.x<self.dims.x and pos.y<self.dims.y:
            return True
        return False
        

    def neighbours(self,pos,t):
        Adjacents = [Point(0,0),Point(-1,0),Point(1,0),Point(0,-1),Point(0,1)]
        neighbours = []
        for a in Adjacents:
            n = pos+a
            if self.inbounds(n) and self.isfree(n,t):
                neighbours.append(n)
        return neighbours


    def walkinthedark(self,st=None,ex=None,t0=0):
        from math import lcm
        ppcm = lcm(self.dims.x,self.dims.y)

        if st is None:
            st = self.start
        if ex is None:
            ex = self.exit
        queue = [(0+Manhattan(st,ex),t0,st)]
        visited = set()
        while queue:
            cost,t,pos = heappop(queue)
            visited.add((t%ppcm,pos))
            if pos==ex:
                return t,pos
            else:
                for next in self.neighbours(pos,t+1):
                    cost = t+1+Manhattan(ex,next)
                    if ((t+1)%ppcm,next) not in visited and (cost,t+1,next) not in queue:
                        heappush(queue,(cost,t+1,next))


    def toarray(self,t=0):
        arr = np.zeros((self.dims.y,self.dims.x),dtype=int)
        for p in self.north:
            pt  = (p -Point(0,t))%self.dims
            arr[pt.y,pt.x] += 1
        for p in self.south:
            pt  = (p +Point(0,t))%self.dims
            arr[pt.y,pt.x] += 1
        for p in self.east:
            pt  = (p +Point(t,0))%self.dims
            arr[pt.y,pt.x] += 1
        for p in self.west:
            pt  = (p -Point(t,0))%self.dims
            arr[pt.y,pt.x] += 1
        return arr

In [147]:
valley = Valley.fromfile("input24.txt")

In [148]:
exvalley = Valley.fromfile("ex24.txt")

In [153]:
tex,ex = exvalley.walkinthedark(exvalley.start,exvalley.exit,t0=0)
print(f"One way: {tex}")
tst,st = exvalley.walkinthedark(exvalley.exit,exvalley.start,t0=tex)
tex,ex = exvalley.walkinthedark(exvalley.start,exvalley.exit,t0=tst)
print(f"With the snack: {tex}")

One way: 18
With the snack: 54


In [154]:
tex,ex = valley.walkinthedark(valley.start,valley.exit,t0=0)
print(f"One way: {tex}")
tst,st = valley.walkinthedark(valley.exit,valley.start,t0=tex)
tex,ex = valley.walkinthedark(valley.start,valley.exit,t0=tst)
print(f"With the snack: {tex}")

One way: 247
With the snack: 728


In [ ]:
for t in range(10):
    print(t)
    plt.imshow(valley.toarray(t),vmax=4)
    plt.show()